In [6]:
import requests 
import pandas as pd

In [1]:
# ONTARIO SUPPLY
URL = "https://www.ieso.ca/ieso/api/PowerData/MarketAtAGlance?"
r = requests.get(url=URL)
data = r.json()
supply_data = data['Subset'][2]

keys_to_ignore = ['Generator Availability', 'Generator Availability at Peak', 'Hourly Imports', 'Hourly Exports', 'Maximum Ontario Import Capacity']
supply_data['Information']
current_supply = {}
for info in supply_data['Information']:
  key = info['InfoTitle']['Value']
  if key not in keys_to_ignore:
    value = info['NewVal']
    if value == '':
      current_supply[key] = 0
    else: 
      current_supply[key] = int(value)

df = pd.DataFrame.from_dict(current_supply, orient='index', columns=['Output'])
df['Percentage'] = df['Output'] / df['Output'].sum()
df

,Output,Percentage
Nuclear,8469,0.487818
Coal,0,0.000000
Hydro,4065,0.234145
Gas,1226,0.070618
Biofuel,132,0.007603
Solar,8,0.000461
Wind,3461,0.199355


In [24]:
# ONTARIO DEMAND
URL = "https://www.ieso.ca/ieso/api/PowerData/DemandSupplyPrice?"
r = requests.get(url=URL)
data = r.json()

ontario_demand_hourly = [i for i in data['OD_Actual'] if i]

latest_5min_demand = [i for i in data['OD_5min'] if i][-1]

# TODO NOT SURE IF THIS WILL ALWAYS WORK (the len-1 index)
latest_projected_demand = data['OD_Projected'][len(ontario_demand_hourly)-1]
latest_hourly_demand = ontario_demand_hourly[-1]

info = {'Latest 5 Min Demand': [latest_5min_demand], 'Latest Hourly Demand': [latest_hourly_demand], 'Current Projected Demand': [latest_projected_demand]}
df = pd.DataFrame(data=info)
df


,Latest 5 Min Demand,Latest Hourly Demand,Current Projected Demand
0,19636.4,18839.5,19608.0


In [32]:
# ONTARIO PRICE
URL = "https://www.ieso.ca/ieso/api/PowerData/DemandSupplyPrice?"
r = requests.get(url=URL)
data = r.json()

hoep = [i for i in data['PRICE_HOEP'] if i]

# hourly energy price (average of the 12 MCP in each hour)
latest_hoep = hoep[-1]

# TODO NOT SURE IF THIS WILL ALWAYS WORK (the len+1 index)
latest_projected_hoep = data['PRICE_HOEP_Projected'][len(hoep)+1]

# market clearing price (set every 5 minutes)
latest_mcp = [i for i in data['PRICE_MPC'] if i][-1]
# Global Adjustment includes cost of building and maintaining infrastructure
latest_ga = [i for i in data['PRICE_HOEP_GA'] if i][-1]


info = {'Latest HOEP': [latest_hoep], 'Latest MCP': [latest_mcp], 'Latest GA': [latest_ga], 'Current Projected HOEP': [latest_projected_hoep]}
df = pd.DataFrame(data=info)
df


,Latest HOEP,Latest MCP,Latest GA,Current Projected HOEP
0,40.87,41.8,89.16,41.74


In [2]:
# ALBERTA SUPPLY
URL = 'http://ets.aeso.ca/ets_web/ip/Market/Reports/CSDReportServlet'
r = requests.get(url=URL)

# NOTE OTHER IS CONSIDERED BIOFUEL
data = pd.read_html(r.text, match='GENERATION', header=0, skiprows=1)[0]
data.columns=['Group', 'Maximum Capacity', 'Total Net Generation', 'Dispatched Contingency Reserve']
data['Percentage'] = data['Total Net Generation'] / data['Total Net Generation'].iloc[:-1].sum()
data

,Group,Maximum Capacity,Total Net Generation,Dispatched Contingency Reserve,Percentage
0,GAS,9573,5762,43,0.601336
1,HYDRO,894,175,252,0.018263
2,ENERGY STORAGE,50,0,50,0.000000
3,SOLAR,336,1,0,0.000104
4,WIND,2139,1586,0,0.165519
5,OTHER,433,316,12,0.032979
6,DUAL FUEL,540,243,5,0.025360
7,COAL,2530,1499,0,0.156439
8,TOTAL,16495,9582,362,1.000000


In [59]:
# ALBERTA DEMAND AND PRICE
# Aggregated by hour
URL = "https://www.aeso.ca/ets/ets-actual.json"
r = requests.get(url=URL)
data = r.json()

latest_time = data['ActualAIL'][0][0]
latest_demand = data['ActualAIL'][0][1]
for demand_pair in data['ActualAIL']:
    if latest_time < demand_pair[0]:
        latest_time = demand_pair[0]
        latest_demand = demand_pair[1]

latest_price = data['ActualPrice'][-1][1]
for price_pair in data['ActualPrice']:
    if latest_time == price_pair[0]:
        latest_price = price_pair[1]

latest_forecast_price = data['ForecastPrice'][-1][1]
for forecast_price_pair in data['ForecastPrice']:
    if latest_time == forecast_price_pair[0]:
        latest_forecast_price = forecast_price_pair[1]

latest_forecast_demand = data['ForecastAIL'][-1][1]
for forecast_demand_pair in data['ForecastAIL']:
    if latest_time == forecast_demand_pair[0]:
        latest_forecast_demand = forecast_demand_pair[1]

info = {'Latest Price': [latest_price], 'Current Projected Price': [latest_forecast_price], 'Latest Demand': [latest_demand], 'Current Projected Demand': [latest_forecast_demand]}
df = pd.DataFrame(data=info)
df


1642978800000


,Latest Price,Current Projected Price,Latest Demand,Current Projected Demand
0,50.35,54.13,10149,10186


In [3]:
# QUEBEC SUPPLY
URL = "https://www.hydroquebec.com/data/documents-donnees/donnees-ouvertes/json/production.json"
# Other is considered biofuel https://www.hydroquebec.com/documents-data/open-data/electricity-generation-quebec/
translation = {
    'hydraulique': 'hydro',
    'eolien': 'wind',
    'autres': 'biofuel',
    'solaire': 'solar',
    'thermique': 'thermal',
    'total': 'total'
}
r = requests.get(url=URL)
data = r.json()
latest_info = {}
for energy_info in reversed(data['details']):
  if energy_info['valeurs']['total'] != 0.0:
    latest_info = energy_info
    break

latest_info_english = {}
for source_type in latest_info['valeurs']:
  latest_info_english[translation[source_type]] = latest_info['valeurs'][source_type]

df = pd.DataFrame.from_dict(latest_info_english, orient='index', columns=['Output'])

# Reverse sort the dataframe so total is last row
df = df.iloc[::-1]
df['Percentage'] = df['Output'] / df['Output'].iloc[:-1].sum()
df


,Output,Percentage
solar,0.0,0.000000
biofuel,563.0,0.022265
wind,799.0,0.031599
hydro,23924.0,0.946136
total,25286.0,1.000000
